<a href="https://colab.research.google.com/github/chandanadasarii/NLP/blob/master/Spell_Checker_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Spell Checker using nltk** :

This task will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this task you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Step 1 

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import nltk
nltk.download("all")

In [0]:
readfile = open("/content/drive/My Drive/NLP/holbrook.txt", "r")
readlines = readfile.readlines()
rawdata = [line.replace("\n","").split(" ") for line in readlines]
char = "|" 
ind = 0
key1 = 'original'
key2 = 'corrected'
key3 = 'indexes'
data=[]

def correct(rawdata,char):
  
  for sublist in rawdata:
    ind = 0
    original=[]
    corrected=[]
    indexes=[]
    data_dict={}
    for word in sublist:
      if char in word:
        split_list = word.split(char) # Split the word based on '|'
        original.append(split_list[0])
        corrected.append(split_list[1])
        indexes.append(ind)
      else:
        original.append(word)
        corrected.append(word)
      ind = ind+1
      
    data_dict[key1]=original
    data_dict[key2]=corrected
    data_dict[key3]=indexes
    data.append(data_dict)
    
correct(rawdata,char) 


assert(data[2] == {
   'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
   'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
   'indexes': [9]
})

The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [0]:
test = data[:100]
train = data[100:]


## **Step 2** : 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [0]:
from collections import Counter

sent_list=[]
for i in range(0,len(train)):
  sent_list.append(train[i]["corrected"])

word_list=[]
for sentence in sent_list:
  for word in sentence:
    word_list.append(word.lower())
word_dictionary = Counter(word_list) # word_dictionary has the word and its frequency in a <key,value> format


def unigram(word):
    word=word.lower()
    return (word_dictionary[word])
    

def prob(word):
    word=word.lower()
    p = float(unigram(word))/(len(word_list))
    return p

# Test your code with the following
assert(unigram("me")==87)

## **Step 3**: 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [0]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [0]:
from nltk.metrics.distance import edit_distance
dist_dict={}
positions = []

def get_candidates(token):
  min_distance = 1000 #initialization by assuming no word length is >1000 char :P
  for word in word_dictionary:
      dist = edit_distance(token.lower(),word.lower())
      dist_dict[word]=dist #saving the distances of all words which wrt to token in dist_dict<word,distance>
  for w,d in dist_dict.items(): 
      if d == min_distance:
        positions.append(w)
      if d < min_distance:
        min_distance = d
        positions = []
        positions.append(w)
  return positions


# Test your code as follows
assert get_candidates("minde") == ['mind', 'mine']

## Step 4 :

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [0]:
from collections import Counter

def correct(sentence):
  corrected_sentence=[]
  for w in sentence:
    if w.lower() in word_dictionary:
        corrected_sentence.append(w)
    else:
        listw=get_candidates(w)# if word is not present in the dictionary call get_candidates method
        if(len(listw)==1):
            corrected_sentence.append(listw[0]) 
        elif(len(listw)>1):
            dictw = {}
            for i in listw:
                dictw[i] = prob(i)
            c = Counter(dictw)
            listw = max(dictw,key=dictw.get) #extracting the word which has max unigram probability
            corrected_sentence.append(listw)
  
  return corrected_sentence        
    
# print(correct(['NIGEL', 'THRUSH', 'page', '48']))
assert(correct(["this","whitr","cat"]) == ['this','white','cat'])   

## **Step 5** : 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [0]:
import nltk
from nltk.corpus import wordnet as wn

def accuracy(test):
  count=0.0
  Totall=0.0
  for i in range(0,len(test)):
    test_sent_orig = test[i]["original"]
    system_corrected = correct(test_sent_orig)
    Totall= Totall+len(test[i]["corrected"])
    for w,c,o in zip(system_corrected,test[i]["corrected"],test[i]["original"]):
      if(w.lower()==c.lower()):
        count = count+1
  
  print "Total number of words correctly identified :",count ,"out of",Totall
  acc = count/Totall 
  print "Accuracy of this model :" ,(acc*100)
  return (acc)

accuracy(test)


Total number of words correctly identified : 947.0 out of 1129.0
Accuracy of this model : 83.8795394154


0.8387953941541186



---



Overall accuracy with the above system using Unigram probabilities and edit distance is **83.87**. I can clearly notice that many words are misclassified  due to 

1.   Less training data
2.   Tense mismatch or singular/Plural mismatch
       for ex : kill/killed ; saturday/saturdays ; sunday/sundays ;races/race
3.    Named entities which needs a special attentions. for named entities also our model is finding the nearest words and calculating unigram probability. which is meaningless
For example : 
for the word "NIGEL" we are getting "night"
similary for "THRUSH"--through; BBC--abc; I.T.V-tv; Moore--more; Babra;Brinner-dinner; Anglia--again; 

4.   For some words which are already correct as its not present in the training data it is replacing with other nearest word.
for ex: wean with jean; sang with gang; joy with boy; sky with say ; yule with hole

5.   Numerical digits also needs a special attention compared to the other words
6.  **The above model is not capturing the semantics of the sentences which plays vital role for correction of sentence**

Above all are important factors which are effecting  the accuarcy of our sentence correction system. My improved algorithm will systamatically takes care of all the factors mentioned here.  **


---



## **Step 6 :**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lecutures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


### Improvised Algorithm :



**1. Lemmatization :**
* Used lemmatization on words to get its base word and check the new lemmatized word is present in training data or words.words() - Reason for this is words may appear in different forms and there is possibility that they may not be present in training data and wordnet. However, it is valid word. Hence to detect this - I used lemmatization.

code snippet :

from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()



**2. Digit/Number detection : **


* Ignoring the numbers (dont correct the numbers like page number and date etc.). If this is not implemented the number in test data is replaced with most nearest number in training data to avoid this we need to ignore ditigits and dates.

*  With the help of Part of Speech (pos) tagging idntified the numbers/digits.
   
   nltk.pos_tag(sentence) gives the tags for all the words present in the sentence

   if tag(word) == "CD" that means word is a number. 

**3.Named Entity Reconginition : **

*Used nltk library for detecting named entities. This is very important as earlier our correcting algorithm ignored the named entities and got the most nearest word with the help of get_candidates to correct the given word.

For identifying NamedEntities i have used ne_chunk from nltk package.

"isnamed_entity(word)" method which i defined below returns True if it is a named entity else it returns False.

For example :

NIGEL THRUSH page 48", is "night through page 48", because word NIGEL and THRUSH is not present in training data. After implementing of named entities NIGEL THRUSH is recognised as named entities. Hence the predicted sentence is "NIGEL THRUSH page 48".

**4.Proper Noun Identification :

*  With the help of POS tagging identified the named entities which are proper nouns with the help of tag NNP

**5. Valid word or not :**

* Used wordnet.words()  from nltk.corpus package to see if the word is valid word.
Reason for this is words may appear in different forms(past tense/ present tense/ future tense) and there is possibility that they may not be present in training data and wordnet even though it is a valid word. 
    
**6.     Bigram probabilities** :
* Using bigrams and conditional frequency distribution I am trying to capture the semantics of the sentence to one level above.

for ex : bigrams("police","came") = 3 that means these two words occured together 3 times in order.
bigram_prob("police","came")= 0.06 tells that the word "came" conditional probability after the word "police" 
based on this bigrams and bigram probability we can capture the semantics of the sentence.


with all the above additions my new model giving the siginificant improvement over the previous model.

**Accuracy of my new model is : 91.76
Total number of words correctly identified : 1036.0 out of 1129.0**

**Further improvements:**

1. Use of higher order ngrams (for example trigrams), we can capture the sentence semantics better.
2. Collecting large amount of training data and training our model on this data.
3. Considering the grammer of the sentence (for example tense of the sentence): Need to research in this area for advanced techniques.

In [0]:
from nltk.util import ngrams
from nltk.probability import ConditionalFreqDist
# These methods are for getting bigrams and bigram probabilities
sent_list=[]
for i in range(0,len(train)):
  sent_list.append(train[i]["corrected"])
training=[]
for sentence in sent_list:
  for word in sentence:
    training.append(word.lower())
bigrams = nltk.bigrams(training)

cfd = ConditionalFreqDist(bigrams)

def bigrams(prev_word, cur_word):
  count = 0
  if cfd[prev_word].get(cur_word) is not None:
    count = cfd[prev_word].get(cur_word)
  return count

def bigram_prob(prev_word, cur_word):
  return float(bigrams(prev_word, cur_word))/(sum(cfd[prev_word].values())+1)





In [0]:
from nltk import ne_chunk
# this method is for checking named entities
def isnamed_entity(word):
  word_temp=[]
  word_temp.append(word)
  entity = nltk.ne_chunk(nltk.pos_tag(word_temp))
  for chunk in entity:
    if hasattr(chunk,"label"):
      return True
  return False

In [0]:
# Improvised method for sentence correction 
import nltk
from collections import Counter
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import wordnet as wn
  
def correct_new(sentence):
    corrected_sentence = []
    sentence_postag = nltk.pos_tag(sentence)
    index = 0
    for (w,tag) in sentence_postag:
      if tag == "NNP" or tag == "CD" or isnamed_entity(w.lower()) or(lemmatizer.lemmatize(w.lower()) in word_dictionary) or (lemmatizer.lemmatize(w.lower()) in wn.words()) or (w.lower() in word_dictionary) or w.lower() in wn.words():
            corrected_sentence.append(w)
      else:
            listw=get_candidates(w)
            if index < len(sentence):
              next_tag = sentence[index+1]
              index += 1
            else:
              next_tag = sentence[index]  
            if(len(listw)==1):
                corrected_sentence.append(listw[0])
            elif(len(listw)>1):
                dictw = {}
                for i in listw:
                    dictw[i] = bigram_prob(i, next_tag)
                c=Counter(dictw)
                listw = max(dictw,key=dictw.get)
                corrected_sentence.append(listw)
  
                
    return corrected_sentence



## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4

In [0]:
import nltk
from nltk.corpus import wordnet as wn

def accuracy_new(test):
    count=0.0
    Totall=0.0
    for i in range(0,len(test)):
      test_sent_orig = test[i]["original"]
      system_corrected = correct_new(test_sent_orig)
      Totall= Totall+len(test[i]["corrected"])
      for w,c,o in zip(system_corrected,test[i]["corrected"],test[i]["original"]):
        if(w.lower()==c.lower()):
          count = count+1
    print "Total number of words correctly identified :",count ,"out of",Totall
    acc = count/Totall 
    print "Accuracy of this model :" ,(acc*100)
    return (acc)

print(accuracy_new(test))
             
          


Total number of words correctly identified : 1036.0 out of 1129.0
Accuracy of this model : 91.7626217892
0.917626217892


## Results

**Earlier Algorithm:**

Accuracy : 93.87

Total number of words correctly identified : 947.0 out of 1129.0

**New Improvised Algorithm:**

Accuracy : 91.76

Total number of words correctly identified : 1036.0 out of 1129.0